In [2]:
import sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()  # for plot styling
from tabulate import tabulate

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D # for 3D plots

De A

General: DIRECTORIO (identificador de la vivienda)

Condiciones de la zona de vivienda: COD_UPZ(upz)

De B

Factores ambientales: NVCBP15D (contaminación del aire), NVCBP15F (generacion y manejo inadecuado de basuras), NVCBP11C (hay alcantarillado)
NVCBP15I(contaminacion de agua)

Condiciones de la zona de vivienda: NVCBP14A(cerca a fabricas), NVCBP15H(insectos => rural), NVCBP1(estado de la via), NVCBP14B(cerca de basureros)

Condiciones del hogar: NVCBP11AA(estrato), NVCBP8A(humedad), NVCBP8A(hay ventilación), NVCBP11B(hay acueducto)
NVCBP13(material de los pisos)

Alergenos:

Salud

De C

Condiciones del hogar: NHCCP36A (hay lavamanos) NHCCP36C (hay tanque de reserva)

De F

General: NPCFP14F (tiene enfermedades respiratorias)

Salud:NPCFP13A (por lo menos una vez al año va al medico)

In [15]:
# df2021 = pd.read_csv('Proyecto 2 - BI - Asma DatosAjus/Datos_proyecto_II_BI_2021.csv', encoding="ISO-8859-1", delimiter=',')
# df2017 = pd.read_csv('Proyecto 2 - BI - Asma DatosAjus/Datos_proyecto_II_BI_2017.csv', encoding="ISO-8859-1", delimiter=',')
# dfs2021 = pd.read_csv('DatosMultipropositoSinAsma-Proy2/Datos_proyecto_II_BI_2021_sin_asma.csv', encoding="ISO-8859-1", delimiter=',')
# dfs2017 = pd.read_csv('DatosMultipropositoSinAsma-Proy2/Datos_proyecto_II_BI_2017_sin_asma.csv', encoding="ISO-8859-1", delimiter=',')
dfC2017 = pd.read_csv('data2017/Condiciones habitacionales del hogar   (Capitulo C)/Condiciones habitacionales del hogar   (Capitulo C).csv', encoding="ISO-8859-1", delimiter=';')
dfC2021 = pd.read_csv('data2021/Condiciones habitacionales del hogar (Capitulo C)/Condiciones habitacionales del hogar (Capítulo C).csv', encoding="ISO-8859-1", delimiter=';')


/var/folders/c3/74tsbvcd7f59g59y1gww3qk40000gn/T/ipykernel_9061/3397493431.py:5: DtypeWarning: Columns (68,69,70,123,124) have mixed types. Specify dtype option on import or set low_memory=False.
  dfC2017 = pd.read_csv('data2017/Condiciones habitacionales del hogar   (Capitulo C)/Condiciones habitacionales del hogar   (Capitulo C).csv', encoding="ISO-8859-1", delimiter=';')
/var/folders/c3/74tsbvcd7f59g59y1gww3qk40000gn/T/ipykernel_9061/3397493431.py:6: DtypeWarning: Columns (4,25,30,47) have mixed types. Specify dtype option on import or set low_memory=False.
  dfC2021 = pd.read_csv('data2021/Condiciones habitacionales del hogar (Capitulo C)/Condiciones habitacionales del hogar (Capítulo C).csv', encoding="ISO-8859-1", delimiter=';')


In [9]:
dfC2017.shape



(109111, 137)

In [10]:
dfC2017.shape

(109111, 137)

In [11]:
dfs2021.shape

NameError: name 'dfs2021' is not defined

In [16]:
dfs2017.shape

(315225, 552)

In [12]:
display(dfC2017.sample(5))

,ï»¿DIRECTORIO_HOG,DIRECTORIO,SECUENCIA_P,ORDEN,NHCCP1,NHCCP2,NHCCP3,NHCCP4A,NHCCP4B,NHCCP4C,...,NHCCP46E,NHCCP46F,NHCCP46G,NHCCP46H,NHCCP46I,NHCCP46J,NHCCP46K,NHCCP46L,NHCCP46M,FEX_C
5515,1094301,109430,1,1,2,1500000,1,1,,,...,3,4,5,5,20,30,30,20,30,"97,345142304"
77709,2438991,243899,1,1,1,,1,,1,,...,5,2,2,20,20,30,20,20,30,"32,059934447"
18128,1365801,136580,1,1,1,,2,,,,...,30,5,5,120,30,90,90,30,30,"16,542605559"
81039,2503011,250301,1,1,1,,1,1,,,...,20,10,5,15,15,20,30,5,60,"89,764255589"
72618,2348551,234855,1,1,1,,1,1,,,...,5,5,5,5,5,5,5,5,5,"17,68478686"


In [13]:
display(df2017.sample(5))

,Unnamed: 0,DIRECTORIO,DIRECTORIO_HOG,DIRECTORIO_PER,SECUENCIA,SECUENCIA_P,ORDEN,NVCAP99,DPTOMPIO,CLASE,...,NPCHP34,NPCHP34A,NPCHP35A,NPCHP35B,NPCHP35C,NPCHP35D,NPCHP35E,NPCHP35F,NPCHP35J,NPCHP35I
2944,2945,191064,1910641,19106411,1,1,1,04NOV2017:00:00:00,25662,1,...,NaN,NaN,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
3415,3416,231646,2316461,23164611,1,1,1,20NOV2017:00:00:00,25662,1,...,NaN,NaN,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
3781,3782,248734,2487341,24873412,1,1,1,21SEP2017:00:00:00,11001,1,...,NaN,NaN,1.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0
3303,3304,227631,2276311,22763112,1,1,1,11NOV2017:00:00:00,11001,1,...,NaN,NaN,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
3528,3529,235285,2352851,23528511,1,1,1,21NOV2017:00:00:00,11001,1,...,NaN,NaN,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0


In [17]:
display(dfs2021.sample(5))

,Unnamed: 0,DIRECTORIO,DIRECTORIO_HOG,DIRECTORIO_PER,DPTO,MPIO,CLASE,COD_LOCALIDAD,NOMBRE_LOCALIDAD,COD_UPZ_GRUPO,...,NPCHP32A,NPCHP33,NPCHP34,NPCHP35A,NPCHP35B,NPCHP35C,NPCHP35D,NPCHP35E,NPCHP35F,NPCHP35I
215308,215309,326637.0,3266371,32663712,11,11001,1,18.0,Rafael Uribe Uribe,55.0,...,NaN,NaN,NaN,1.0,2.0,2.0,2.0,2.0,2.0,2.0
207849,207850,3006387.0,30063871,300638711,11,11001,1,1.0,Usaquen,14.0,...,NaN,NaN,NaN,2.0,2.0,2.0,2.0,2.0,2.0,2.0
176775,176776,1870934.0,18709341,187093411,25,25785,1,NaN,NaN,NaN,...,NaN,NaN,NaN,2.0,2.0,2.0,2.0,2.0,2.0,2.0
243689,243690,340521.0,3405211,34052112,11,11001,1,5.0,Usme,52.0,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
169461,169462,1169407.0,11694071,116940712,11,11001,1,16.0,Puente Aranda,41.0,...,NaN,NaN,NaN,2.0,2.0,2.0,2.0,2.0,2.0,2.0


In [18]:
display(dfs2017.sample(5))

,Unnamed: 0,DIRECTORIO,DIRECTORIO_HOG,DIRECTORIO_PER,SECUENCIA,SECUENCIA_P,ORDEN,NVCAP99,DPTOMPIO,CLASE,...,NPCHP34,NPCHP34A,NPCHP35A,NPCHP35B,NPCHP35C,NPCHP35D,NPCHP35E,NPCHP35F,NPCHP35J,NPCHP35I
172210,172211,187271,1872711,18727112,1,1,1,10NOV2017:00:00:00,25260,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15753,15754,109156,1091561,10915611,1,1,1,17SEP2017:00:00:00,11001,1,...,NaN,NaN,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
43958,43959,130935,1309351,13093511,1,1,1,09OCT2017:00:00:00,25754,1,...,NaN,NaN,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
294089,294090,289139,2891391,28913915,1,1,1,06DEC2017:00:00:00,25513,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
293832,293833,287940,2879401,28794011,1,1,1,30NOV2017:00:00:00,11001,1,...,NaN,NaN,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
